# Comparative Analysis
In this notebook, we evaluate ClaSS as a Flink window operator.

In [1]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import numpy as np

import pandas as pd

pd.set_option('display.max_rows', None)

import seaborn as sns

sns.set_theme()
sns.set_color_codes()

import sys

sys.path.append("../")

Let's load the scores from the different methods.

In [3]:
df = pd.read_csv("../experiments/flink_runtimes.csv")
df

,name,dataset,runtime,throughput
0,UTSA,Cane,17.787611,300.208956
1,UTSA,DutchFactory,23.432260,373.886265
2,UTSA,EEGRat,14.872977,134.472073
3,UTSA,EEGRat2,19.371916,103.242239
4,UTSA,Fetal2013,19.310362,932.142012
5,UTSA,GrandMalSeizures,22.048343,835.981206
6,UTSA,GrandMalSeizures2,18.550346,562.415374
7,UTSA,GreatBarbet1,15.743814,298.529956
8,UTSA,GreatBarbet2,15.708935,299.192779
9,UTSA,InsectEPG1,19.033804,893.200313


Let's calculate the mean/median/std throughput.

In [4]:
np.round(df.throughput.mean(axis=0), 0)

1004.0

In [5]:
np.round(df.throughput.median(axis=0), 0)

928.0

In [6]:
np.round(df.throughput.std(axis=0), 0)

310.0

In [7]:
np.round(df.throughput.max(axis=0), 0)

2063.0